In [1]:
%pip install datasets
%pip install transformers accelerate bitsandbytes

import json
from datasets import Dataset
import transformers
print(transformers.__version__)


# Load your kenya_facts.json file
with open("kenya_facts.json", "r") as f:
    data = json.load(f)

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.1)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\Sharon\Desktop\Sharon\My coding\flutter_apps\safari_chat_project\safari_chat_backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.53.3


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)



In [12]:
def format_instruction(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(format_instruction)


Map: 100%|██████████| 2/2 [00:00<00:00, 84.21 examples/s]


In [13]:
%pip install peft trl


from peft import get_peft_model, LoraConfig, TaskType
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling


# LoRA config
# LoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # for LLaMA or similar
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)  

# Training args
training_args = TrainingArguments(
    output_dir="./llama-kenyan-facts",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_dir="./logs",
    save_strategy="no",
    num_train_epochs=1,
    warmup_steps=10,
    logging_steps=10,
    fp16=False, 
    save_total_limit=2,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [14]:
trainer.train()


c:\Users\Sharon\Desktop\Sharon\My coding\flutter_apps\safari_chat_project\safari_chat_backend\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=9, training_loss=1.9561203850640192, metrics={'train_runtime': 539.7901, 'train_samples_per_second': 0.017, 'train_steps_per_second': 0.017, 'total_flos': 28633341100032.0, 'train_loss': 1.9561203850640192, 'epoch': 1.0})